# Common Target and Window

This example demonstrates creating a Target and Window object and using them to submit observationst to both LCO and ESO faciltiies.

In [1]:
from datetime import datetime, timedelta

## Create Python objects for common data
We define out Window and Target objects here. Note that in an automated pipeline, these objects may be created as the result of, for example, a transiet alert. We define them statically here for demonstration purposes.

In [2]:
from aeonlib.models import SiderealTarget, Window

In [3]:
# Create a SiderealTarget - note the format of ra/dec can be any format that Astropy.Angle accepts
target = SiderealTarget(name="Tarantula Nebula", type="ICRS", ra="05h05m38.0s", dec="−69° 05.7′")
print(target.ra, target.dec)

5h05m38s -69d05m42s


In [4]:
# Window bounds can be specified as datetimes or Astropy.time.Time objects
window = Window(start=datetime.now() + timedelta(days=1), end=datetime.now() + timedelta(days=30))
print(window.start, window.end)

2025-05-08 11:35:01.252888 2025-06-06 11:35:01.252896


## Initialize the LCO Facility and create a Request Group
The LCO facility is one of many facilities available in Aeonlib. Here we demonstrate creating a request to the LCO facility and submitting it. This test is done using test credentials. See the README for instructions on configuring each facility.

In [5]:
# We first import the OCS request models
from aeonlib.ocs import Constraints, Location, Request, RequestGroup

# Then we import the LCO instrument we are interested in
from aeonlib.ocs.lco.instruments import Lco1M0ScicamSinistro

In [6]:
lco_request = RequestGroup(
    name="test",
    observation_type="NORMAL",
    operator="SINGLE",
    proposal="TEST_PROPOSAL",
    ipp_value=1.0,
    requests=[
        Request(
            location=Location(telescope_class="1m0"),
            configurations=[
                Lco1M0ScicamSinistro(
                    type="EXPOSE",
                    target=target,  # Our target defined at the top of the file
                    constraints=Constraints(),
                    instrument_configs=[
                        Lco1M0ScicamSinistro.config_class(
                            exposure_count=1,
                            exposure_time=10,
                            mode="central_2k_2x2",
                            optical_elements=Lco1M0ScicamSinistro.optical_elements_class(
                                filter="B"
                            ),
                        )
                    ],
                    acquisition_config=Lco1M0ScicamSinistro.acquisition_config_class(
                        mode="OFF"
                    ),
                    guiding_config=Lco1M0ScicamSinistro.guiding_config_class(
                        mode="ON", optional=True
                    ),
                )
            ],
            windows=[window],  # Our window defined at the top of the file
        )
    ],
)

In [7]:
# Import the LCO Facility and create the facility Object
from aeonlib.ocs.lco.facility import LcoFacility

lco = LcoFacility()

In [8]:
# Validate the request group
valid, errors = lco.validate_request_group(lco_request)
if not valid:
    print("Online validation failed. Server response: %s", errors)
print("Request Valid:", valid)

Request Valid: True


In [9]:
# Now submit the request.
# submit_request_group returns a RequestGroup object that will have state populated.
lco_request = lco.submit_request_group(lco_request)
print(f"LCO request id: {lco_request.id}")
print(f"LCO request state: {lco_request.state}")

LCO request id: 48
LCO request state: PENDING


## Create an ESO Observation Block and Validate
Aeonlib provides an interface to the ESO phase 2 API and tool. ESO works differently than LCO, but we can still use the same Target and Windw objects (remember, in practice these will be dynamic values!) to observe with ESO. Note that ESO requires logging in to their Phase 2 tool online to do a final submission of the block.

In [10]:
# Use the ESO tutorial container ID
ESO_TUTORIAL_CONTAINER_ID = 1538878

# Import the ESO facility and required models
from aeonlib.eso.facility import EsoFacility
from aeonlib.eso.models import AbsoluteTimeConstraint, AbsoluteTimeConstraints

In [11]:
# Create the eso client and follow the phase 2 workflow:
eso = EsoFacility()
folder = eso.create_folder(ESO_TUTORIAL_CONTAINER_ID, "AEONlib.notebook")

# Create an observation block
ob = eso.create_ob(folder, "AEONLIB.notebook.ob")

In [12]:
# Populate the target fields using our target object
ob.target.use_sidereal_target(target)  # Target we defined above
print(ob.target)

dec=np.str_('-69:05:42.000') differential_dec=0.0 differential_ra=0.0 epoch=2000 equinox='J2000' name='Tarantula Nebula' proper_motion_dec=0 proper_motion_ra=0 ra=np.str_('5:05:38.000')


In [13]:
# Populate the rest of the block fields
ob.constraints.name = "My hardest constraints ever"
ob.constraints.airmass = 2.3
ob.constraints.sky_transparency = "Variable, thin cirrus"
ob.constraints.fli = 0.1
ob.constraints.seeing = 2.0
eso.save_ob(ob)

# Add some instrument templates
acq_template = eso.create_template(ob, "UVES_blue_acq_slit")
sci_template = eso.create_template(ob, "UVES_blue_obs_exp")

In [14]:
# Save time window, using our previously defined window
time_constraints = AbsoluteTimeConstraints(
    constraints=[AbsoluteTimeConstraint.from_window(window)]
)
eso.save_absolute_time_constraints(ob, time_constraints)
print(ob.constraints)

airmass=2.3 fli=0.1 moon_distance=30 name='My hardest constraints ever' seeing=2.0 sky_transparency='Variable, thin cirrus' twilight=0 water_vapour=30.0


In [15]:
# Verify our observation block with ESO
errors, success = eso.verify(ob, False)
ob = eso.get_ob(ob.ob_id)
print(f"Errors: {errors}")
print(f"Success: {success}")
print(f"Status: {ob.ob_status}")

Errors: []
Success: True
Status: P


In [16]:
# Clean up the example data.
eso.delete_template(ob, acq_template)
ob = eso.get_ob(ob.ob_id)
eso.delete_template(ob, sci_template)
ob = eso.get_ob(ob.ob_id)
eso.delete_ob(ob)
# Need to refresh the container
folder = eso.get_container(folder.container_id)
eso.delete_container(folder)

# ESO blocks must be set for final approval using the ESO P2 tool.